In [1]:
import pandas as pd
import seaborn as sns
from utils import get_results
import matplotlib.pyplot as plt
from constants import *
import numpy as np
import os

plt.rcParams['figure.dpi'] = 120

In [62]:
def headways(trips, rt_dir, date_, scenario, stop_seq):
    dt_date = pd.to_datetime(date_).date()
    tr = trips[
        trips['departure_sec'].dt.date == dt_date].copy()
    tr = tr[tr['scenario']==scenario]
    tr = tr[(tr['stop_sequence']==stop_seq) & 
            (tr['rt_dir']==rt_dir)]
    tr = tr.sort_values(by='departure_sec').reset_index(drop=True)
    tr['hw_td'] = tr['departure_sec'].diff()
    tr['hw'] = tr['hw_td'].dt.total_seconds()
    cols = ['route_id','trip_id', 'block_id', 'hw_td', 'hw', 
            'arrival_sec', 'departure_sec', 'schd_sec']
    return tr[cols]

def barplots(r, title=None, ylabel=None):
    fig, ax = plt.subplots()
    r['wait_time'] = r['wait_time'].round("S")
    r['wait_time_mins']= r['wait_time'].dt.total_seconds()/60
    r['rt_dir'] = r['route_id'].astype(str) + '-' +r['direction']
    sns.barplot(r, y='wait_time_mins', x='rt_dir', 
                hue='scenario', palette=['black', 'gray', 'darkcyan'],
                ax=ax)
    if title:
        ax.set_title(title)
    if ylabel:
        ax.set_ylabel(ylabel)
    ax.set_xlabel('')
    # plt.ylim(0,20)

def sub_pax(pax):
    df = pax.copy()
    df['wt'] = (df['boarding_time'] - df['arrival_time']).dt.total_seconds()/60
    df['date'] = df['arrival_time'].dt.date
    df['h'] = df['arrival_time'].dt.hour
    df['jt'] = (df['alighting_time'] - df['arrival_time']).dt.total_seconds()/60
    df = df[df['h'].isin([6,7,8])]
    return df

def outbound_only(df, rt_dirs):
    df = df.copy()
    df['rt_dir'] = df['route_id'].astype(str) + '-' + df['direction']    
    df = df[df['rt_dir'].isin(rt_dirs)].copy()    
    return df

def rider_stats(df):
    group_by = ['date', 'scenario']
    ## wait time
    mean_wt = df.groupby(group_by)['wt'].mean().reset_index()
    mean_wt = mean_wt.rename(columns={'wt':'wt_mean'})
    high_wt = df.groupby(group_by)['wt'].quantile(0.9).reset_index()
    high_wt = high_wt.rename(columns={'wt': 'wt_high'})

    wt = mean_wt.merge(high_wt, on=group_by)
    
    ##rbt 
    median_jt = df.groupby(group_by)['jt'].median().reset_index()
    median_jt = median_jt.rename(columns={'jt': 'jt_median'})
    high_jt = df.groupby(group_by)['jt'].mean().reset_index()
    high_jt = high_jt.rename(columns={'jt': 'jt_high'})
    
    jt = median_jt.merge(high_jt, on=group_by)
    jt['rbt'] = jt['jt_high'] - jt['jt_median']
    rbt = jt.drop(labels=['jt_median', 'jt_high'], axis=1)
    
    rider_stats = wt.merge(rbt, on=group_by)
    
    return rider_stats

In [10]:
tstamp = '0709-1246'
path_from_cwd = 'data/sim_out/experiments_' + tstamp

pax = pd.read_csv(os.path.join(path_from_cwd, 'pax.csv'), 
                  parse_dates=['arrival_time', 'boarding_time', 'alighting_time'])
trips = pd.read_csv(os.path.join(path_from_cwd, 'trips.csv'), 
                    parse_dates=['arrival_sec', 'departure_sec', 'schd_sec'])
events = pd.read_csv(os.path.join(path_from_cwd, 'events.csv'))
wlog = pd.read_csv(os.path.join(path_from_cwd, 'worklog.csv'))
slog = pd.read_csv(os.path.join(path_from_cwd, 'superlog.csv'))

In [54]:
terminal_rt_dirs = []
for rt in ROUTES:
    terminal_rt_dirs.append(rt+'-'+OUTBOUND_DIRECTIONS[rt])

In [55]:
pdf = sub_pax(pax)

In [59]:
rider_stats(pdf)

,date,scenario,wt_mean,wt_high,rbt
0,2022-11-21,EHD,8.488635,19.308333,1.883868
1,2022-11-21,EHD-DI,8.362325,18.028333,1.680791
2,2022-11-21,NC,8.844067,20.358333,2.221926
3,2022-11-23,EHD,7.221277,14.786667,1.463710
4,2022-11-23,EHD-DI,7.044816,14.050000,1.769605
5,2022-11-23,NC,6.924250,13.856667,1.394942
6,2022-12-07,EHD,7.205015,14.800000,2.117250
7,2022-12-07,EHD-DI,7.081933,14.133333,1.712161
8,2022-12-07,NC,7.217007,14.493333,1.662531


In [61]:
pdf_out = outbound_only(pdf, terminal_rt_dirs)
rider_stats(pdf_out)

,date,scenario,wt_mean,wt_high,rbt
0,2022-11-21,EHD,7.771984,16.473333,1.617031
1,2022-11-21,EHD-DI,7.401074,14.920000,1.039986
2,2022-11-21,NC,8.502146,19.771667,1.913300
3,2022-11-23,EHD,7.122199,14.566667,1.312827
4,2022-11-23,EHD-DI,6.811079,13.653333,1.711093
5,2022-11-23,NC,6.728208,13.391667,1.531349
6,2022-12-07,EHD,6.536574,13.086667,1.786195
7,2022-12-07,EHD-DI,6.592521,12.950000,1.304244
8,2022-12-07,NC,6.471403,13.018333,1.747819


In [50]:
wlog[wlog['filled']==1].groupby('date')['trip_id'].count().reset_index()

,date,trip_id
0,2022-11-21,6
1,2022-12-07,2


In [72]:
tdf = trips.copy()
tdf['rt_dir'] = tdf['route_id'].astype(str) + '-' + tdf['direction']

day = '2022-11-23'
lst_data = []

for rt_dir in terminal_rt_dirs:
    for scenario in ['NC', 'EHD', 'EHD-DI']:
        hws = headways(tdf, rt_dir, day ,scenario, 1)
        mean_hw = np.round(hws['hw'].mean(), decimals=1)
        sd_hw = np.round(hws['hw'].std(), decimals=1)
        lst_data.append([scenario, rt_dir, mean_hw, sd_hw, round(sd_hw/mean_hw, 2)])
pd.DataFrame(lst_data, columns=['scenario', 'rt_dir', 'mean_hw', 'sd_hw', 'cv'])

,scenario,rt_dir,mean_hw,sd_hw,cv
0,NC,81-East,722.2,313.0,0.43
1,EHD,81-East,819.8,318.6,0.39
2,EHD-DI,81-East,803.9,273.9,0.34
3,NC,91-South,747.9,207.0,0.28
4,EHD,91-South,752.6,130.4,0.17
5,EHD-DI,91-South,752.3,119.1,0.16
6,NC,92-East,661.0,138.0,0.21
7,EHD,92-East,674.2,137.3,0.20
8,EHD-DI,92-East,662.3,136.5,0.21


In [73]:
tdf = trips.copy()
tdf['rt_dir'] = tdf['route_id'].astype(str) + '-' + tdf['direction']

day = '2022-12-07'
lst_data = []

for rt_dir in terminal_rt_dirs:
    for scenario in ['NC', 'EHD', 'EHD-DI']:
        hws = headways(tdf, rt_dir, day ,scenario, 1)
        mean_hw = np.round(hws['hw'].mean(), decimals=1)
        sd_hw = np.round(hws['hw'].std(), decimals=1)
        lst_data.append([scenario, rt_dir, mean_hw, sd_hw, round(sd_hw/mean_hw, 2)])
pd.DataFrame(lst_data, columns=['scenario', 'rt_dir', 'mean_hw', 'sd_hw', 'cv'])

,scenario,rt_dir,mean_hw,sd_hw,cv
0,NC,81-East,610.3,275.7,0.45
1,EHD,81-East,688.8,274.8,0.40
2,EHD-DI,81-East,650.8,215.5,0.33
3,NC,91-South,909.6,358.4,0.39
4,EHD,91-South,911.6,283.6,0.31
5,EHD-DI,91-South,912.6,280.6,0.31
6,NC,92-East,660.5,143.8,0.22
7,EHD,92-East,672.0,133.9,0.20
8,EHD-DI,92-East,709.4,156.7,0.22


In [79]:
headways(tdf, '91-South', '2022-12-07', 'NC', 1)

,route_id,trip_id,block_id,hw_td,hw,arrival_sec,departure_sec,schd_sec
0,91,11565020,652000002631,NaT,NaN,2022-12-07 05:12:00,2022-12-07 05:12:00,2022-12-07 05:12:00
1,91,11129020,652000002632,0 days 00:27:27,1647.0,2022-12-07 05:39:20,2022-12-07 05:39:27,2022-12-07 05:39:00
2,91,1600020,652000002541,0 days 00:13:14,794.0,2022-12-07 05:52:30,2022-12-07 05:52:41,2022-12-07 05:52:30
3,91,24452020,652000002561,0 days 00:21:05,1265.0,2022-12-07 06:13:30,2022-12-07 06:13:46,2022-12-07 06:13:30
4,91,28414020,652000002595,0 days 00:08:51,531.0,2022-12-07 06:22:30,2022-12-07 06:22:37,2022-12-07 06:22:30
5,91,16130020,652000002545,0 days 00:09:29,569.0,2022-12-07 06:32:00,2022-12-07 06:32:06,2022-12-07 06:32:00
6,91,39674020,652000002631,0 days 00:09:36,576.0,2022-12-07 06:41:30,2022-12-07 06:41:42,2022-12-07 06:41:30
7,91,38441020,652000002581,0 days 00:19:35,1175.0,2022-12-07 07:01:00,2022-12-07 07:01:17,2022-12-07 07:01:00
8,91,48610020,652000002632,0 days 00:09:54,594.0,2022-12-07 07:11:00,2022-12-07 07:11:11,2022-12-07 07:11:00
9,91,37199020,652000002541,0 days 00:21:39,1299.0,2022-12-07 07:32:30,2022-12-07 07:32:50,2022-12-07 07:32:30


In [77]:
headways(tdf, '91-South', '2022-12-07', 'EHD-DI', 1)

,route_id,trip_id,block_id,hw_td,hw,arrival_sec,departure_sec,schd_sec
0,91,11565020,652000002631,NaT,NaN,2022-12-07 05:12:00,2022-12-07 05:12:00,2022-12-07 05:12:00
1,91,11129020,652000002632,0 days 00:27:32,1652.0,2022-12-07 05:39:17,2022-12-07 05:39:32,2022-12-07 05:39:00
2,91,1600020,652000002541,0 days 00:17:15,1035.0,2022-12-07 05:56:30,2022-12-07 05:56:47,2022-12-07 05:52:30
3,91,24452020,652000002561,0 days 00:17:21,1041.0,2022-12-07 06:14:08,2022-12-07 06:14:08,2022-12-07 06:13:30
4,91,28414020,652000002595,0 days 00:08:56,536.0,2022-12-07 06:23:04,2022-12-07 06:23:04,2022-12-07 06:22:30
5,91,16130020,652000002545,0 days 00:09:22,562.0,2022-12-07 06:32:17,2022-12-07 06:32:26,2022-12-07 06:32:00
6,91,39674020,652000002631,0 days 00:13:17,797.0,2022-12-07 06:45:30,2022-12-07 06:45:43,2022-12-07 06:41:30
7,91,38441020,652000002581,0 days 00:13:00,780.0,2022-12-07 06:58:21,2022-12-07 06:58:43,2022-12-07 07:01:00
8,91,48610020,652000002632,0 days 00:16:36,996.0,2022-12-07 07:15:00,2022-12-07 07:15:19,2022-12-07 07:11:00
9,91,37199020,652000002541,0 days 00:20:01,1201.0,2022-12-07 07:34:54,2022-12-07 07:35:20,2022-12-07 07:32:30


In [85]:
day = pd.to_datetime('2022-12-07').date()
block_id = 652000002541
scenario = 'EHD-DI'

trips[(trips['block_id'] == block_id) & 
      (trips['scenario']==scenario) & 
      (trips['stop_sequence']==1) & 
      (trips['arrival_sec'].dt.date == day)]

,stop_id,stop_sequence,arrival_sec,departure_sec,passenger_load,ons,offs,schd_sec,trip_id,block_id,run_id,direction,route_id,scenario
18408,14103,1,2022-12-07 05:56:30,2022-12-07 05:56:47,6,6,0,2022-12-07 05:52:30,1600020,652000002541,F169,South,91,EHD-DI
18473,9617,1,2022-12-07 06:35:30,2022-12-07 06:35:30,0,0,0,2022-12-07 06:35:30,17504020,652000002541,F169,North,91,EHD-DI
18539,14103,1,2022-12-07 07:34:54,2022-12-07 07:35:20,10,10,0,2022-12-07 07:32:30,37199020,652000002541,F169,South,91,EHD-DI
18604,9617,1,2022-12-07 08:27:36,2022-12-07 08:27:46,2,2,0,2022-12-07 08:21:30,19868020,652000002541,F169,North,91,EHD-DI


In [88]:
day = pd.to_datetime('2022-12-07').date()
trip_id = 17504020
scenario = 'EHD-DI'

trips[(trips['trip_id'] == trip_id) & 
      (trips['scenario']==scenario) & 
      (trips['arrival_sec'].dt.date == day)].iloc[-3:]

,stop_id,stop_sequence,arrival_sec,departure_sec,passenger_load,ons,offs,schd_sec,trip_id,block_id,run_id,direction,route_id,scenario
18536,17064,64,2022-12-07 07:22:25,2022-12-07 07:22:25,16,0,0,2022-12-07 07:15:31,17504020,652000002541,F169,North,91,EHD-DI
18537,17038,65,2022-12-07 07:24:06,2022-12-07 07:24:30,4,0,12,2022-12-07 07:16:36,17504020,652000002541,F169,North,91,EHD-DI
18538,14103,66,2022-12-07 07:24:54,2022-12-07 07:24:54,0,0,4,2022-12-07 07:17:00,17504020,652000002541,F169,North,91,EHD-DI


In [89]:
slog

,time,lend_rt,borrow_rt,ratio,est_return,req_return
0,2022-11-21 05:46:40,92,81,1.800748,2022-11-21 07:16:00,2022-11-21 07:20:00
1,2022-11-21 06:36:25,81,91,3.706342,2022-11-21 08:14:00,2022-11-21 08:23:00
2,2022-11-21 08:25:30,92,91,2.214256,2022-11-21 09:59:00,2022-11-21 10:13:30
3,2022-11-21 05:46:40,92,81,1.800748,2022-11-21 07:16:00,2022-11-21 07:20:00
4,2022-11-21 06:36:25,81,91,3.706342,2022-11-21 08:14:00,2022-11-21 08:23:00
5,2022-11-21 08:25:30,92,91,2.214256,2022-11-21 09:59:00,2022-11-21 10:13:30
6,2022-12-07 05:45:01,92,81,1.234011,2022-12-07 07:16:00,2022-12-07 07:20:00
7,2022-11-21 05:46:40,92,81,1.800748,2022-11-21 07:16:00,2022-11-21 07:20:00
8,2022-11-21 06:36:25,81,91,3.706342,2022-11-21 08:14:00,2022-11-21 08:23:00
9,2022-11-21 08:25:30,92,91,2.214256,2022-11-21 09:59:00,2022-11-21 10:13:30
